TODO
jason briner - long term perspectives of greenland ice sheet position
- if current approx margin configuration is where it _generally_ is during interglacials then it would *make sense* that there are lots of overdeepenings just up-ice.
- Lindbäck, K., R. Pettersson, A. L. Hubbard, S. H. Doyle, D. van As, A. B. Mikkelsen, and A. A. Fitzpatrick. 2015. Subglacial water drainage, storage and piracy beneath the Greenland ice sheet. Geophysical Research Letters 42:7606–14.
see lindback references in ross et al., 2018
- Lindbäck, K., R. Petterson, S. H. Doyle, C. Helanow, P. Jansson, S. S. Kristensen, L. Stenseng, R. Forsberg, and A. L. Hubbard. 2014. High-resolution ice thickness and bed topography of a land-terminating section of the Greenland ice sheet. Earth System Science Data 6:331–38.
is there evidence of *preferential* erosion signature

lake massu
simon mudd, mikael attal - what are lakes? 
rivers **cannot** erode below base level.

TODO
send PN velocity figures and summary paragraph

In [1]:
from dask.distributed import Client
import pandas as pd
import itslive
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import box
import matplotlib.pyplot as plt
import seaborn as sns
import imagery
import xrspatial as xrs
import numpy as np
import xarray as xr
from tqdm import tqdm

In [170]:
client = Client()
print(f'dask dashboard: {client.dashboard_link}')

dask dashboard: http://127.0.0.1:50288/status


In [171]:
import importlib
import imagery
import utils
import get_velocity
_ = importlib.reload(utils)
_ = importlib.reload(get_velocity)
_ = importlib.reload(imagery)

In [ ]:
# boxes = gpd.read_file('../data/terminus_boxes.geojson')
sites = gpd.read_file('../data/potential_study_sites_v2.geojson',)
sites['velocity'] = pd.Series(dtype=object)
lines = gpd.read_file('../data/streams_v2.geojson')

In [ ]:
V = {}
V[1] = get_velocity.CentreLiner(geo=lines.loc[1, 'geometry'],
                                buff_dist=2_000,
                                index=1)

In [ ]:
streams = []
for k, v in V.items():
    streams.append(v.tidy_stream)
sites['streams'] = gpd.GeoSeries(streams, crs=3413)
sites['streams'] = sites['streams'].astype(sites['geometry'].dtype)

stream_gdf = gpd.GeoDataFrame(geometry=streams, crs=3413)
# stream_gdf.to_file('../data/streams_v2.geojson')
stream_gdf.explore()

In [ ]:
import cmcrameri.cm as cmc
def v_profile_plotter(self, ax):
    _annual_v = (self.v_line_df.groupby(
        ['cumul_dist', 'year']
        )['v'].agg(['median',
                    partial(median_abs_deviation,
                            nan_policy='omit')])
        .reset_index())
    
    _annual_v['y1'] = (_annual_v['median']
                        + (1.4826 * _annual_v['median_abs_deviation'])
    )
    _annual_v['y2'] = (_annual_v['median']
                        - (1.4826 * _annual_v['median_abs_deviation'])
    )
    norm = Normalize(*_annual_v['year'].agg(['min', 'max']))
    cmap = cmc.batlow_r

    for year in _annual_v['year'].unique():
        _idx = _annual_v['year']==year
        ax.plot(_annual_v.loc[_idx, 'cumul_dist']/1000,
                _annual_v.loc[_idx, 'median'],
                c=cmap(norm(year)))
        
        ax.fill_between(_annual_v.loc[_idx, 'cumul_dist']/1000,
                        _annual_v.loc[_idx, 'y1'],
                        _annual_v.loc[_idx, 'y2'],
                        alpha=0.2,
                        color=cmap(norm(year)),
                        label=year)
    plt.colorbar(ScalarMappable(cmap=cmap, norm=norm), ax=ax)
    
    ax.set(ylim=(0,None),
           xlabel=('distance from terminus (km)'),
           ylabel=('velocity (m/yr)'))   



In [ ]:
plotter(V[1])

- look at underlying dodgy velocity images
TODO look at GRIMpP and MEASUREs

TODO
- PN thinks that relative change is sufficient to *control* for variations in glacier geometry (read: width/size).
- most interested in *relative* change of velocity over final few kilometers.
    - i.e. 

- it all comes from the same raw processed v data

## imagery

In [ ]:
study_sites = gpd.read_file('../data/potential_study_sites_v1.geojson')
study_sites = study_sites.loc[study_sites['notes'].isin(['yes', 'maybe'])]
study_sites.reset_index(drop=True, inplace=True)
study_sites_5k = study_sites.buffer(5_000).to_crs(4326)

In [ ]:
items = imagery.get_annual_median_mosaic(
    study_sites.loc[0,'geometry'],
    buffer_dist=15_000,
    src_crs=study_sites.crs,
    target_crs=4326,
    timeperiod='2023-01-01/2024-01-01',
    months=[8,9],
    )

In [ ]:
(xrs.multispectral.true_color(
    *items.squeeze()
    .transpose('band', 'y', 'x'))
 .plot.imshow(rgb='band')
)

In [ ]:
import stackstac
stackstac.stack(items, assets=['B04', 'B03', 'B02'])

In [ ]:
xrs.multispectral.true_color(*img.squeeze().transpose('band', 'y', 'x'),
                             nodata=np.nan).plot.imshow(rgb='band')

In [ ]:
xrs.multispectral.true_color(*img.squeeze().transpose('band', 'y', 'x'),
                             nodata=np.nan).plot.imshow(rgb='band')

In [ ]:
z0 = itslive.velocity_cubes.find_by_polygon(study_sites_5k.geometry[0])
zarr_bounds = Polygon(z0[0]['geometry']['coordinates'][0])

In [ ]:
m = study_sites_5k.loc[[0]].explore()
gpd.GeoSeries([zarr_bounds], crs=4326).explore(m=m)